In [16]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set the dataset folder path (go up one level from notebooks to find dataset)
dataset_folder = Path('../dataset')  # Note the ../ to go up one folder level

print(" Loading Campus Security Dataset...")
print("=" * 60)

# Load all CSV files
profiles = pd.read_csv(dataset_folder / 'student or staff profiles.csv')
card_swipes = pd.read_csv(dataset_folder / 'campus card_swipes.csv')
wifi_logs = pd.read_csv(dataset_folder / 'wifi_associations_logs.csv')
library_checkouts = pd.read_csv(dataset_folder / 'library_checkouts.csv')
lab_bookings = pd.read_csv(dataset_folder / 'lab_bookings.csv')
cctv_frames = pd.read_csv(dataset_folder / 'cctv_frames.csv')
face_embeddings = pd.read_csv(dataset_folder / 'face_embeddings.csv')
helpdesk_notes = pd.read_csv(dataset_folder / 'free_text_notes (helpdesk or RSVPs).csv')

print(" All datasets loaded successfully!\n")

# Display summary statistics
datasets = {
    'Student Profiles': profiles,
    'Card Swipes': card_swipes,
    'WiFi Logs': wifi_logs,
    'Library Checkouts': library_checkouts,
    'Lab Bookings': lab_bookings,
    'CCTV Frames': cctv_frames,
    'Face Embeddings': face_embeddings,
    'Helpdesk Notes': helpdesk_notes
}

print(" Dataset Summary:")
print("-" * 60)
for name, df in datasets.items():
    print(f"{name:.<30} {len(df):>8,} records, {len(df.columns):>3} columns")

print("\n Total records across all datasets:", f"{sum(len(df) for df in datasets.values()):,}")


 Loading Campus Security Dataset...
 All datasets loaded successfully!

 Dataset Summary:
------------------------------------------------------------
Student Profiles..............    7,000 records,  10 columns
Card Swipes...................    8,000 records,   3 columns
WiFi Logs.....................    8,000 records,   3 columns
Library Checkouts.............    7,000 records,   4 columns
Lab Bookings..................    7,000 records,   6 columns
CCTV Frames...................    7,000 records,   4 columns
Face Embeddings...............    6,973 records,   2 columns
Helpdesk Notes................    7,000 records,   5 columns

 Total records across all datasets: 57,973


In [17]:
# View first few rows of each dataset
print(" STUDENT PROFILES - Sample Data:")
print(profiles.head(3))
print("\n Columns:", list(profiles.columns))
print("\n" + "="*80)

print("\n CARD SWIPES - Sample Data:")
print(card_swipes.head(3))
print("\n Columns:", list(card_swipes.columns))
print("\n" + "="*80)

print("\n WIFI LOGS - Sample Data:")
print(wifi_logs.head(3))
print("\n Columns:", list(wifi_logs.columns))
print("\n" + "="*80)

print("\n LIBRARY CHECKOUTS - Sample Data:")
print(library_checkouts.head(3))
print("\n Columns:", list(library_checkouts.columns))


 STUDENT PROFILES - Sample Data:
  entity_id        name     role             email department student_id  \
0   E100000  Neha Mehta  student  user0@campus.edu      CIVIL     S39256   
1   E100001  Neha Kumar  student  user1@campus.edu    Physics     S14165   
2   E100002  Neha Singh  student  user2@campus.edu      Admin     S13478   

  staff_id card_id     device_hash  face_id  
0      NaN   C3286  DH6d0bd80c8f8e  F100000  
1      NaN   C1488  DH75af13047587  F100001  
2      NaN   C4257  DH7a5f80cb6e8d  F100002  

 Columns: ['entity_id', 'name', 'role', 'email', 'department', 'student_id', 'staff_id', 'card_id', 'device_hash', 'face_id']


 CARD SWIPES - Sample Data:
  card_id  location_id            timestamp
0   C7151          GYM  2025-09-13 14:02:40
1   C4562  ADMIN_LOBBY  2025-09-22 00:16:08
2   C6840      LIB_ENT  2025-09-12 22:50:40

 Columns: ['card_id', 'location_id', 'timestamp']


 WIFI LOGS - Sample Data:
      device_hash     ap_id        timestamp
0  DH6034d82be28b  AP

In [18]:
# ===== ENTITY RESOLUTION SYSTEM =====
print(" ENTITY RESOLUTION SYSTEM")
print("="*80)

# Create entity mapping dictionary
entity_map = {}

# Map all identifiers from profiles to entity_id
for _, row in profiles.iterrows():
    entity_id = row['entity_id']
    
    # Map each identifier type to the entity
    if pd.notna(row['student_id']):
        entity_map[row['student_id']] = entity_id
    if pd.notna(row['staff_id']):
        entity_map[row['staff_id']] = entity_id
    if pd.notna(row['card_id']):
        entity_map[row['card_id']] = entity_id
    if pd.notna(row['device_hash']):
        entity_map[row['device_hash']] = entity_id
    if pd.notna(row['face_id']):
        entity_map[row['face_id']] = entity_id
    if pd.notna(row['email']):
        entity_map[row['email']] = entity_id

print(f" Mapped {len(entity_map):,} identifiers to {len(profiles):,} entities")
print(f"   Average identifiers per entity: {len(entity_map)/len(profiles):.1f}")

# Show sample mapping
print(f"\n Sample Entity Mapping (first entity):")
first_entity = profiles.iloc[0]['entity_id']
first_entity_ids = [k for k, v in entity_map.items() if v == first_entity]
print(f"   Entity: {first_entity}")
print(f"   Linked IDs: {first_entity_ids}")


 ENTITY RESOLUTION SYSTEM
 Mapped 30,585 identifiers to 7,000 entities
   Average identifiers per entity: 4.4

 Sample Entity Mapping (first entity):
   Entity: E100000
   Linked IDs: ['S39256', 'C3286', 'DH6d0bd80c8f8e', 'F100000', 'user0@campus.edu']


In [19]:
# ===== LINK ALL DATASETS TO ENTITIES =====
print("\n LINKING DATASETS TO ENTITIES")
print("="*80)

# Add entity_id to card swipes (via card_id)
card_swipes['entity_id'] = card_swipes['card_id'].map(entity_map)
print(f" Card Swipes: {card_swipes['entity_id'].notna().sum():,}/{len(card_swipes):,} linked")

# Add entity_id to wifi logs (via device_hash)
wifi_logs['entity_id'] = wifi_logs['device_hash'].map(entity_map)
print(f" WiFi Logs: {wifi_logs['entity_id'].notna().sum():,}/{len(wifi_logs):,} linked")

# Library checkouts already have entity_id
print(f" Library Checkouts: {library_checkouts['entity_id'].notna().sum():,}/{len(library_checkouts):,} linked")

# Add entity_id to lab bookings
lab_bookings['entity_id'] = lab_bookings['booking_id'].str.extract(r'(E\d+)')[0]
print(f" Lab Bookings: {lab_bookings['entity_id'].notna().sum():,}/{len(lab_bookings):,} linked")

# Add entity_id to CCTV frames (via face_id)
cctv_frames['entity_id'] = cctv_frames['face_id'].map(entity_map)
print(f" CCTV Frames: {cctv_frames['entity_id'].notna().sum():,}/{len(cctv_frames):,} linked")

# Add entity_id to face embeddings (via face_id)
face_embeddings['entity_id'] = face_embeddings['face_id'].map(entity_map)
print(f" Face Embeddings: {face_embeddings['entity_id'].notna().sum():,}/{len(face_embeddings):,} linked")

# Helpdesk notes - extract entity from note_id or use entity_id column
if 'entity_id' in helpdesk_notes.columns:
    print(f" Helpdesk Notes: {helpdesk_notes['entity_id'].notna().sum():,}/{len(helpdesk_notes):,} linked")
else:
    helpdesk_notes['entity_id'] = helpdesk_notes['note_id'].str.extract(r'(E\d+)')[0]
    print(f" Helpdesk Notes: {helpdesk_notes['entity_id'].notna().sum():,}/{len(helpdesk_notes):,} linked")

print("\n All datasets successfully linked to entities!")



 LINKING DATASETS TO ENTITIES
 Card Swipes: 8,000/8,000 linked
 WiFi Logs: 8,000/8,000 linked
 Library Checkouts: 7,000/7,000 linked
 Lab Bookings: 0/7,000 linked
 CCTV Frames: 4,213/7,000 linked
 Face Embeddings: 0/6,973 linked
 Helpdesk Notes: 7,000/7,000 linked

 All datasets successfully linked to entities!


In [20]:
# ===== TIMELINE GENERATION SYSTEM (CORRECTED) =====
def generate_entity_timeline(entity_id):
    """Generate complete timeline for a given entity"""
    
    timeline = []
    
    # 1. Card Swipes
    entity_swipes = card_swipes[card_swipes['entity_id'] == entity_id].copy()
    for _, row in entity_swipes.iterrows():
        timeline.append({
            'timestamp': pd.to_datetime(row['timestamp'], errors='coerce'),
            'event_type': 'physical_access',
            'location': row['location_id'],
            'source': 'card_swipe',
            'details': f"Card swipe at {row['location_id']}"
        })
    
    # 2. WiFi Connections (uses 'ap_id' not 'location_id')
    entity_wifi = wifi_logs[wifi_logs['entity_id'] == entity_id].copy()
    for _, row in entity_wifi.iterrows():
        timeline.append({
            'timestamp': pd.to_datetime(row['timestamp'], errors='coerce'),
            'event_type': 'network_access',
            'location': row['ap_id'],  # ← Changed from location_id
            'source': 'wifi_log',
            'details': f"WiFi connected at {row['ap_id']}"
        })
    
    # 3. Library Checkouts
    entity_library = library_checkouts[library_checkouts['entity_id'] == entity_id].copy()
    for _, row in entity_library.iterrows():
        timeline.append({
            'timestamp': pd.to_datetime(row['timestamp'], errors='coerce'),
            'event_type': 'library_transaction',
            'location': 'Library',
            'source': 'library_system',
            'details': f"Checked out book {row['book_id']}"
        })
    
    # 4. Lab Bookings (uses 'room_id' and 'start_time')
    entity_labs = lab_bookings[lab_bookings['entity_id'] == entity_id].copy()
    for _, row in entity_labs.iterrows():
        timeline.append({
            'timestamp': pd.to_datetime(row['start_time'], errors='coerce'),  # ← Changed from timestamp
            'event_type': 'lab_booking',
            'location': row['room_id'],  # ← Changed from location_id
            'source': 'lab_system',
            'details': f"Lab booked: {row['room_id']}"
        })
    
    # 5. CCTV Detections
    entity_cctv = cctv_frames[cctv_frames['entity_id'] == entity_id].copy()
    for _, row in entity_cctv.iterrows():
        timeline.append({
            'timestamp': pd.to_datetime(row['timestamp'], errors='coerce'),
            'event_type': 'face_recognition',
            'location': row['location_id'],
            'source': 'cctv_system',
            'details': f"Face detected at {row['location_id']}"
        })
    
    # Convert to DataFrame and clean
    timeline_df = pd.DataFrame(timeline)
    
    if len(timeline_df) > 0:
        timeline_df = timeline_df[timeline_df['timestamp'].notna()]
        timeline_df = timeline_df.sort_values('timestamp').reset_index(drop=True)
    
    return timeline_df

# Test with first entity
print(" TIMELINE GENERATION TEST (UPDATED)")
print("="*80)

test_entity = profiles.iloc[0]['entity_id']
test_name = profiles.iloc[0]['name']

timeline = generate_entity_timeline(test_entity)

print(f"\n Timeline for {test_name} ({test_entity}):")
print(f"   Total events: {len(timeline):,}")

if len(timeline) > 0:
    print(f"   Date range: {timeline['timestamp'].min()} to {timeline['timestamp'].max()}")
    print(f"   Event types: {timeline['event_type'].value_counts().to_dict()}")
    
    print(f"\n First 10 events:")
    display_cols = ['timestamp', 'event_type', 'location', 'details']
    print(timeline[display_cols].head(10).to_string(index=False))
else:
    print("    No events found for this entity")


 TIMELINE GENERATION TEST (UPDATED)

 Timeline for Neha Mehta (E100000):
   Total events: 6
   Date range: 2025-08-29 07:41:18 to 2025-09-19 00:36:50
   Event types: {'physical_access': 5, 'face_recognition': 1}

 First 10 events:
          timestamp       event_type    location                   details
2025-08-29 07:41:18  physical_access HOSTEL_GATE Card swipe at HOSTEL_GATE
2025-09-02 13:58:39  physical_access     LIB_ENT     Card swipe at LIB_ENT
2025-09-04 02:31:00 face_recognition     LAB_101  Face detected at LAB_101
2025-09-05 06:46:02  physical_access  AUDITORIUM  Card swipe at AUDITORIUM
2025-09-12 19:18:02  physical_access  AUDITORIUM  Card swipe at AUDITORIUM
2025-09-19 00:36:50  physical_access     LAB_101     Card swipe at LAB_101


In [21]:
# ===== ANOMALY DETECTION =====
def detect_anomalies(entity_id, timeline_df):
    """Detect unusual patterns in entity timeline"""
    
    anomalies = []
    
    if len(timeline_df) == 0:
        return anomalies
    
    # 1. Late night activity (11 PM - 5 AM)
    timeline_df['hour'] = timeline_df['timestamp'].dt.hour
    late_night = timeline_df[(timeline_df['hour'] >= 23) | (timeline_df['hour'] <= 5)]
    if len(late_night) > 0:
        anomalies.append({
            'type': 'late_night_activity',
            'severity': 'medium',
            'count': len(late_night),
            'details': f"{len(late_night)} activities between 11 PM - 5 AM"
        })
    
    # 2. Weekend activity
    timeline_df['day_of_week'] = timeline_df['timestamp'].dt.dayofweek
    weekend = timeline_df[timeline_df['day_of_week'].isin([5, 6])]
    if len(weekend) > 10:  # More than 10 weekend events
        anomalies.append({
            'type': 'excessive_weekend_activity',
            'severity': 'low',
            'count': len(weekend),
            'details': f"{len(weekend)} activities on weekends"
        })
    
    # 3. Rapid location changes (< 5 minutes apart)
    timeline_df = timeline_df.sort_values('timestamp')
    timeline_df['time_diff'] = timeline_df['timestamp'].diff().dt.total_seconds() / 60
    rapid_changes = timeline_df[(timeline_df['time_diff'] < 5) & (timeline_df['time_diff'] > 0)]
    if len(rapid_changes) > 5:
        anomalies.append({
            'type': 'rapid_location_changes',
            'severity': 'high',
            'count': len(rapid_changes),
            'details': f"{len(rapid_changes)} location changes within 5 minutes"
        })
    
    # 4. Inactive periods (no activity for > 48 hours)
    timeline_df['gap'] = timeline_df['timestamp'].diff().dt.total_seconds() / 3600
    long_gaps = timeline_df[timeline_df['gap'] > 48]
    if len(long_gaps) > 0:
        anomalies.append({
            'type': 'long_inactivity',
            'severity': 'medium',
            'count': len(long_gaps),
            'details': f"{len(long_gaps)} gaps of >48 hours without activity"
        })
    
    return anomalies

# Test anomaly detection
anomalies = detect_anomalies(test_entity, timeline)

print("\n ANOMALY DETECTION RESULTS")
print("="*80)
print(f"Detected {len(anomalies)} anomalies for {test_name}:\n")

for i, anomaly in enumerate(anomalies, 1):
    print(f"{i}. [{anomaly['severity'].upper()}] {anomaly['type']}")
    print(f"   {anomaly['details']}\n")



 ANOMALY DETECTION RESULTS
Detected 2 anomalies for Neha Mehta:

1. [MEDIUM] late_night_activity
   2 activities between 11 PM - 5 AM

2. [MEDIUM] long_inactivity
   3 gaps of >48 hours without activity



In [22]:
# ===== SYSTEM-WIDE ANALYTICS (CORRECTED) =====
print(" CAMPUS SECURITY SYSTEM - COMPLETE ANALYTICS")
print("="*80)

# 1. Overall Statistics
total_entities = len(profiles)
total_events = (len(card_swipes) + len(wifi_logs) + len(library_checkouts) + 
                len(lab_bookings) + len(cctv_frames))

print(f"\n Overall System Statistics:")
print(f"   Total Entities: {total_entities:,}")
print(f"   Total Events Tracked: {total_events:,}")
print(f"   Average Events per Entity: {total_events/total_entities:.1f}")

# Safe date range
date_min = pd.to_datetime(card_swipes['timestamp'], errors='coerce').min()
date_max = pd.to_datetime(card_swipes['timestamp'], errors='coerce').max()
print(f"   Date Range: {date_min} to {date_max}")

# 2. Event Distribution
print(f"\n Event Distribution by Type:")
event_counts = {
    'Card Swipes': len(card_swipes),
    'WiFi Connections': len(wifi_logs),
    'Library Checkouts': len(library_checkouts),
    'Lab Bookings': len(lab_bookings),
    'CCTV Detections': len(cctv_frames)
}
for event_type, count in sorted(event_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / total_events) * 100
    print(f"   {event_type:.<25} {count:>8,} ({percentage:>5.1f}%)")

# 3. Most Active Locations (using correct column names)
print(f"\n Most Active Locations:")
all_locations = pd.concat([
    card_swipes['location_id'],      # Card swipes use location_id
    wifi_logs['ap_id'],               # WiFi uses ap_id
    lab_bookings['room_id'],          # Lab bookings use room_id
    cctv_frames['location_id']        # CCTV uses location_id
]).value_counts().head(10)

for location, count in all_locations.items():
    print(f"   {location:.<25} {count:>8,} events")

# 4. Activity by Hour
print(f"\n Activity Distribution by Hour:")
card_swipes_copy = card_swipes.copy()
card_swipes_copy['hour'] = pd.to_datetime(card_swipes_copy['timestamp'], errors='coerce').dt.hour
hourly_activity = card_swipes_copy['hour'].value_counts().sort_index()

max_count = hourly_activity.max() if len(hourly_activity) > 0 else 1

for hour in range(24):
    count = hourly_activity.get(hour, 0)
    bar_length = int((count / max_count) * 50) if max_count > 0 else 0
    bar = '█' * bar_length
    print(f"   {hour:02d}:00  {bar} {count:>5,}")

print("\n Analytics complete!")


 CAMPUS SECURITY SYSTEM - COMPLETE ANALYTICS

 Overall System Statistics:
   Total Entities: 7,000
   Total Events Tracked: 37,000
   Average Events per Entity: 5.3
   Date Range: 2025-08-27 00:05:10 to 2025-09-25 23:51:54

 Event Distribution by Type:
   Card Swipes..............    8,000 ( 21.6%)
   WiFi Connections.........    8,000 ( 21.6%)
   Library Checkouts........    7,000 ( 18.9%)
   Lab Bookings.............    7,000 ( 18.9%)
   CCTV Detections..........    7,000 ( 18.9%)

 Most Active Locations:
   AUDITORIUM...............    2,918 events
   LAB_305..................    2,840 events
   LAB_101..................    2,788 events
   HOSTEL_GATE..............    1,907 events
   ADMIN_LOBBY..............    1,893 events
   LIB_ENT..................    1,884 events
   GYM......................    1,868 events
   CAF_01...................    1,856 events
   LAB_102..................    1,021 events
   SEM_01...................    1,020 events

 Activity Distribution by Hour:
   0

In [10]:
# Check actual column names in each dataset
print("Column check:")
print("WiFi logs columns:", list(wifi_logs.columns))
print("Lab bookings columns:", list(lab_bookings.columns))


Column check:
WiFi logs columns: ['device_hash', 'ap_id', 'timestamp', 'entity_id']
Lab bookings columns: ['booking_id', 'entity_id', 'room_id', 'start_time', 'end_time', 'attended (YES/NO)']


In [23]:
# ===== SCAN ALL ENTITIES FOR ANOMALIES =====
print("\n ANOMALY DETECTION - FULL SYSTEM SCAN")
print("="*80)

all_anomalies = []

# Scan first 50 entities (to save time - you can increase this later)
sample_entities = profiles.head(50)

print(f"Scanning {len(sample_entities)} entities for anomalies...")

for idx, entity_row in sample_entities.iterrows():
    entity_id = entity_row['entity_id']
    entity_name = entity_row['name']
    
    # Generate timeline
    timeline = generate_entity_timeline(entity_id)
    
    # Detect anomalies
    if len(timeline) > 0:
        anomalies = detect_anomalies(entity_id, timeline)
        
        if len(anomalies) > 0:
            all_anomalies.append({
                'entity_id': entity_id,
                'name': entity_name,
                'anomaly_count': len(anomalies),
                'anomalies': anomalies
            })

print(f"\n Scanned {len(sample_entities)} entities")
print(f" Found {len(all_anomalies)} entities with anomalies")

# Show top 10 entities with most anomalies
print(f"\n Top 10 Entities with Most Anomalies:")
sorted_anomalies = sorted(all_anomalies, key=lambda x: x['anomaly_count'], reverse=True)[:10]

for i, entity_data in enumerate(sorted_anomalies, 1):
    print(f"\n{i}. {entity_data['name']} ({entity_data['entity_id']})")
    print(f"   Total Anomalies: {entity_data['anomaly_count']}")
    for anomaly in entity_data['anomalies']:
        print(f"   • [{anomaly['severity'].upper()}] {anomaly['type']}: {anomaly['details']}")

print("\n" + "="*80)



 ANOMALY DETECTION - FULL SYSTEM SCAN
Scanning 50 entities for anomalies...

 Scanned 50 entities
 Found 46 entities with anomalies

 Top 10 Entities with Most Anomalies:

1. Neha Mehta (E100000)
   Total Anomalies: 2
   • [MEDIUM] late_night_activity: 2 activities between 11 PM - 5 AM
   • [MEDIUM] long_inactivity: 3 gaps of >48 hours without activity

2. Neha Kumar (E100001)
   Total Anomalies: 2
   • [MEDIUM] late_night_activity: 3 activities between 11 PM - 5 AM
   • [MEDIUM] long_inactivity: 2 gaps of >48 hours without activity

3. Neha Singh (E100002)
   Total Anomalies: 2
   • [MEDIUM] late_night_activity: 2 activities between 11 PM - 5 AM
   • [MEDIUM] long_inactivity: 2 gaps of >48 hours without activity

4. Rohan Desai (E100004)
   Total Anomalies: 2
   • [MEDIUM] late_night_activity: 3 activities between 11 PM - 5 AM
   • [MEDIUM] long_inactivity: 3 gaps of >48 hours without activity

5. Neha Rao (E100005)
   Total Anomalies: 2
   • [MEDIUM] late_night_activity: 1 activitie

In [24]:
# ===== EXPORT RESULTS =====
print("\n EXPORTING RESULTS")
print("="*80)

# Create output folder
import os
output_folder = Path('../output')
os.makedirs(output_folder, exist_ok=True)

# 1. Export entity profiles with linked identifiers
profiles.to_csv(output_folder / 'entity_profiles.csv', index=False)
print(f" Exported: entity_profiles.csv ({len(profiles):,} records)")

# 2. Export sample timeline
sample_timeline = generate_entity_timeline(profiles.iloc[0]['entity_id'])
sample_timeline.to_csv(output_folder / 'sample_timeline.csv', index=False)
print(f" Exported: sample_timeline.csv ({len(sample_timeline):,} records)")

# 3. Export anomaly report
anomaly_report = []
for entity_data in all_anomalies:
    for anomaly in entity_data['anomalies']:
        anomaly_report.append({
            'entity_id': entity_data['entity_id'],
            'name': entity_data['name'],
            'anomaly_type': anomaly['type'],
            'severity': anomaly['severity'],
            'details': anomaly['details']
        })

if len(anomaly_report) > 0:
    anomaly_df = pd.DataFrame(anomaly_report)
    anomaly_df.to_csv(output_folder / 'anomaly_report.csv', index=False)
    print(f" Exported: anomaly_report.csv ({len(anomaly_df):,} records)")
else:
    print(f" No anomalies to export")

# 4. Export system statistics
stats = {
    'metric': [
        'Total Entities', 
        'Total Events', 
        'Entities with Anomalies',
        'Total Anomaly Count',
        'Scan Coverage'
    ],
    'value': [
        total_entities,
        total_events,
        len(all_anomalies),
        sum(e['anomaly_count'] for e in all_anomalies),
        f"{len(sample_entities)}/{total_entities} entities scanned"
    ]
}
stats_df = pd.DataFrame(stats)
stats_df.to_csv(output_folder / 'system_statistics.csv', index=False)
print(f" Exported: system_statistics.csv")

print(f"\n All results saved to: {output_folder.absolute()}")
print("\n CAMPUS SECURITY SYSTEM ANALYSIS COMPLETE!")
print("="*80)



 EXPORTING RESULTS
 Exported: entity_profiles.csv (7,000 records)
 Exported: sample_timeline.csv (6 records)
 Exported: anomaly_report.csv (75 records)
 Exported: system_statistics.csv

 All results saved to: c:\Users\nitish kumar yadav\Downloads\campus project\notebook\..\output

 CAMPUS SECURITY SYSTEM ANALYSIS COMPLETE!
